A linear regression for weather predict based on tensorflow
===============================================
Using keras
--------------


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math, time, math, random
%matplotlib inline

#### Using tensonflow version: 

In [2]:
tf.__version__

'2.4.1'

## Introduce to keras

> Copyright 2015 The TensorFlow Authors. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at 
> http://www.apache.org/licenses/LICENSE-2.0
> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and

Keras is a deep learning API written in Python, running on top of the machine learning platform TensorFlow. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result as fast as possible is key to doing good research.

Keras is:

Simple -- but not simplistic. Keras reduces developer cognitive load to free you to focus on the parts of the problem that really matter.

Flexible -- Keras adopts the principle of progressive disclosure of complexity: simple workflows should be quick and easy, while arbitrarily advanced workflows should be possible via a clear path that builds upon what you've already learned.

Powerful -- Keras provides industry-strength performance and scalability: it is used by organizations and companies including NASA, YouTube, or Waymo.

## import data sets

using weather data from gsod
and gain data on ftp://ftp.ncdc.noaa.gov/pub/data/gsod

using data randomly choose from year 2021

In [3]:
def read_data(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

## File walker

In [4]:
from os import walk

In [5]:
path = "data_op/722860/"

In [6]:
for sdir, bdir, name in walk(path):
    file_name = name
    print(name)

['722860-23119-1978.op', '722860-23119-1939.op', '722860-23119-1968.op', '722860-23119-1948.op', '722860-23119-1958.op', '722860-23119-1949.op', '722860-23119-1959.op', '722860-23119-1979.op', '722860-23119-1938.op', '722860-23119-1969.op', '722860-23119-1946.op', '722860-23119-1981.op', '722860-23119-1972.op', '722860-23119-2010.op', '722860-99999-2003.op', '722860-23119-1933.op', '722860-23119-1962.op', '722860-23119-1991.op', '722860-23119-1956.op', '722860-23119-1985.op', '722860-23119-1976.op', '722860-23119-1942.op', '722860-23119-2020.op', '722860-23119-2014.op', '722860-23119-1952.op', '722860-23119-1966.op', '722860-23119-1937.op', '722860-23119-1995.op', '722860-23119-1984.op', '722860-23119-1977.op', '722860-23119-1943.op', '722860-23119-2005.op', '722860-23119-2021.op', '722860-23119-2015.op', '722860-23119-1953.op', '722860-23119-1967.op', '722860-23119-1936.op', '722860-23119-1994.op', '722860-23119-1947.op', '722860-23119-1980.op', '722860-23119-1973.op', '722860-23119-2

In [7]:
!cd data_op/ && ls

722860 745700 949999


In [8]:
# data = read_data(path+random.choice(file_name))
data = np.load("./npy/raw_avg_722860.npy")

In [9]:
data.shape

(32308,)

In [10]:
def org_file(data):
    """将温度信息抽取"""
    tem_avg, tem_max, tem_min = [], [], []
    for i in data[1:-1]:
        tem_avg.append(float(i[26:30]))
        tem_max.append(float(i[104:108]))
        tem_min.append(float(i[111:116]))
    return tem_avg

In [11]:
# tem_avg = org_file(data.split("\n"))
tem_avg = data

In [12]:
def generate(train_set, data_set):
    """train_set: training set 的大小，
    data_set: 传入list, 数据本身"""
    test_set = len(data_set) - 8

    def get_data(set1, set2, data_set):
        train_data = []
        train_label = []
        for i in range(set1, set2):
            train_data.append(data_set[i:i + 7])
            train_label.append(data_set[i + 8])
        train_data = np.array(train_data)
        train_label = np.array(train_label)
        return train_data, train_label

    train_data, train_label = get_data(0, train_set, data_set)
    test_data, test_label = get_data(train_set, test_set, data_set)

    return (train_data, train_label), (test_data, test_label)

In [13]:
(train_data, train_label), (test_data, test_label) = generate(int(len(tem_avg)*0.75), tem_avg)

In [14]:
train_label, test_label = train_label.reshape(-1, 1), test_label.reshape(-1, 1)

In [15]:
test = train_data[0]

In [16]:
test = test.reshape(1,7)

In [17]:
def get_train_data(data):
    def polynize(dataset):
        dataset = dataset.reshape(1, -1)
        dataset_copy = dataset.copy()
        for i in range(2,2):
            dataset = np.append(dataset, dataset_copy**i, 1)
        return dataset
    def reshape(target_data, data):
        target_data = np.append((7,1), 0)
        target_data = data.reshape(-1)
        return target_data
        
    i =  0 
    out = np.zeros((data.shape[0], 7))
    for index in data:
        out[i] = reshape(data[i], polynize(index))
        i += 1
    return out

In [18]:
train_data_redo = get_train_data(train_data)
test_data_redo = get_train_data(test_data)

In [19]:
# train_data_redo.dtype = float, train_label.dtype = float

train_data_redo, train_label = tf.convert_to_tensor(train_data_redo/100), tf.convert_to_tensor(train_label/100)

In [20]:
train_label

<tf.Tensor: shape=(24231, 1), dtype=float64, numpy=
array([[0.654],
       [0.618],
       [0.562],
       ...,
       [0.779],
       [0.727],
       [0.709]])>

## tensorflow model

In [21]:
train_data_redo

<tf.Tensor: shape=(24231, 7), dtype=float64, numpy=
array([[0.566, 0.57 , 0.515, ..., 0.646, 0.595, 0.67 ],
       [0.57 , 0.515, 0.589, ..., 0.595, 0.67 , 0.58 ],
       [0.515, 0.589, 0.646, ..., 0.67 , 0.58 , 0.654],
       ...,
       [0.658, 0.628, 0.581, ..., 0.614, 0.67 , 0.718],
       [0.628, 0.581, 0.605, ..., 0.67 , 0.718, 0.772],
       [0.581, 0.605, 0.614, ..., 0.718, 0.772, 0.779]])>

In [26]:
"""model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(7,)),
    tf.keras.layers.Dense(1)
])"""
model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(7,)),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])

In [27]:
model.compile(optimizer="adam",
             loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 7)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 25,857
Trainable params: 25,857
Non-trainable params: 0
__________________________________________________

In [28]:
model.fit(train_data_redo, train_label, batch_size=1000, epochs=1000)

Epoch 1/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.2844
Epoch 2/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0268
Epoch 3/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0133
Epoch 4/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0089
Epoch 5/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0070
Epoch 6/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0059
Epoch 7/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 8/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0047
Epoch 9/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 10/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 11/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0040
Epoch 12/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.0038
Epoch 13/1000
25/25 [====

25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 102/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 103/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 104/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 105/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 106/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 107/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 108/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 109/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 110/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 111/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.0026
Epoch 112/1000
25/25 [==============================] - 0s 4ms/step - loss: 0.0026
Epoch 113/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 200/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 201/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 202/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 203/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 204/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 205/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 206/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 207/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 208/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 209/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 210/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 211/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 298/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 299/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 300/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 301/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 302/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 303/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 304/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 305/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 306/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 307/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 308/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 309/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 396/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 397/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 398/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 399/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 400/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 401/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 402/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 403/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 404/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 405/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 406/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 407/1000
25/2

...

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 592/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 593/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 594/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 595/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 596/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 597/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 598/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 599/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 600/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 601/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 602/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 603/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 690/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 691/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 692/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 693/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 694/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 695/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 696/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 697/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 698/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 699/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 700/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 701/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 788/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 789/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 790/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 791/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 792/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 793/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 794/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 795/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 796/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 797/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 798/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 799/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 886/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 887/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 888/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 889/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 890/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 891/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 892/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 893/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 894/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 895/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 896/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 897/1000
25/2

25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 984/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 985/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 986/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 987/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 988/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 989/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 990/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 991/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 992/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 993/1000
25/25 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 994/1000
25/25 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 995/1000
25/2

In [29]:
model.evaluate(test_data_redo, test_label)

253/253 [==============================] - 0s 672us/step - loss: 56.1010


56.10102081298828

In [25]:
model.predict(train_data_redo[0])

ValueError: in user code:

    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1569 predict_function  *
        return step_function(self, iterator)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    

In [36]:
]

array([[-75.5],
       [-69.3],
       [-57.3],
       [-53.8],
       [-54.3],
       [-60.6],
       [-70.3],
       [-68.8],
       [-74.1],
       [-76.8],
       [-76.8],
       [-76.3],
       [-59.6],
       [-51.5],
       [-52.3],
       [-56.8],
       [-61.2],
       [-69.5],
       [-69.3],
       [-72.9],
       [-72.8],
       [-76.6],
       [-77.3],
       [-78.8],
       [-77.9],
       [-78.3],
       [-79.3],
       [-77.1],
       [-78.9],
       [-77. ],
       [-69.8],
       [-74. ],
       [-63.7],
       [-64.3],
       [-67.6],
       [-72.4],
       [-76.3],
       [-76.8]])